In [2]:
#@title Imports

!pip install pymatgen -q
!pip install rdfpy -q
#!pip install git+https://github.com/diffpy/diffpy.pdffit2.git

import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import math
#from scipy.optimize import curve_fit, minimize
#from scipy.integrate import trapz, simpson
import random
from pymatgen import core
from rdfpy import rdf
import scipy as scipy

from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/CNPEM/Jacutingaita/Analises/PDF/'

Mounted at /content/drive


## **1. Cálculo do S(Q) e G(r)**

In [3]:
#@title 1.1. Classes e funções
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os

class DataProcessor:
    def __init__(self, file_path, lobato_path, start, end, ds, Elements):
        self.file_path = file_path
        self.start = start
        self.end = end
        self.ds = ds
        self.lobato = lobato_path

        total = []
        for element in Elements:
            if Elements[element]:
                total.append(Elements[element][1])
        soma = sum(total)

        for element in Elements:
            if Elements[element]:
                Elements[element].append(Elements[element][1] / soma)
        self.Elements = Elements

        # Load and process data in the constructor
        self.x, self.iq, self.q, self.s, self.s2 = self.load_and_process_data()
        self.lobato_factors = self.calculate_Lobato_Factors()
        self.fq_sq, self.gq, self.fqfit, self.iqfit = self.calculate_fq_gq()
        self.N, self.C, self.autofit = self.calculate_N_and_parameters()

    def load_and_process_data(self):
        data = np.array(pd.read_csv(self.file_path, sep='\t', header=None))
        Iq, x = data[:, 0], np.arange(0, len(data[:, 0]), 1)
        x, iq = x[self.start:self.end], Iq[self.start:self.end]
        q = x * self.ds * 2 * math.pi
        s = q / (2 * math.pi)
        s2 = s ** 2
        return x, iq, q, s, s2

    def calculate_Lobato_Factors(self):
        FACTORS = []
        Lobato_Factors = np.empty(shape=(0))
        df = pd.read_csv(self.lobato, header=None)

        for element in self.Elements:
            if self.Elements[element]:
                FACTORS.append(np.array(df.iloc[self.Elements[element][0]]))

        for LF in FACTORS:
            Lobato_Factors = np.append(Lobato_Factors,
                                       (((LF[0] * (self.s2 * LF[5] + 2) / (self.s2 * LF[5] + 1) ** 2)) +
                                        ((LF[1] * (self.s2 * LF[6] + 2) / (self.s2 * LF[6] + 1) ** 2)) +
                                        ((LF[2] * (self.s2 * LF[7] + 2) / (self.s2 * LF[7] + 1) ** 2)) +
                                        ((LF[3] * (self.s2 * LF[8] + 2) / (self.s2 * LF[8] + 1) ** 2)) +
                                        ((LF[4] * (self.s2 * LF[9] + 2) / (self.s2 * LF[9] + 1) ** 2))))
        Lobato_Factors = Lobato_Factors.reshape(2, len(self.x))

        return Lobato_Factors

    def calculate_fq_gq(self):
        fq = np.empty(shape=(0))
        gq = np.empty(shape=(0))

        print("Elements:", self.Elements)
        print("Length of Lobato_Factors:", len(self.lobato_factors))

        for i in range(0, len(self.lobato_factors)):
            if self.Elements[i + 1]:
                fq = np.append(fq, self.lobato_factors[i] * self.Elements[i + 1][2])
                gq = np.append(gq, (self.lobato_factors[i] ** 2) * self.Elements[i + 1][2])

        fq_sq = np.sum(fq.reshape(2, len(self.x)), axis=0)
        fq_sq = fq_sq ** 2
        gq = np.sum(gq.reshape(2, len(self.x)), axis=0)
        fqfit = gq[self.end - (self.start+1)]
        iqfit = self.iq[self.end - (self.start+1)]

        return fq_sq, gq, fqfit, iqfit

    def calculate_N_and_parameters(self):
        wi = np.ones_like(self.x)

        a1 = np.sum(wi * self.gq * self.iq)
        a2 = np.sum(wi * self.iq * self.fqfit)
        a3 = np.sum(wi * self.gq * self.iqfit)
        a4 = np.sum(wi) * self.fqfit * self.iqfit
        a5 = np.sum(wi * self.gq ** 2)
        a6 = 2 * np.sum(wi * self.gq) * self.fqfit
        a7 = np.sum(wi) * self.fqfit * self.fqfit

        N = (a1 - a2 - a3 + a4) / (a5 - a6 + a7)

        # Fitting Parameters
        C = self.iqfit - N * self.fqfit
        autofit = N * self.gq + C

        return N, C, autofit

    def calculate_SQ_PhiQ(self, iq, damping):
        sq = (((iq - self.autofit)) / (self.N * self.fq_sq)) + 1
        fq = (((iq - self.autofit) * self.s) / (self.N * self.fq_sq)) * np.exp(-self.s2 * damping)

        return sq, fq

    def calculate_Gr(self, fq, rmax, dr):
        Gr = np.zeros_like(self.q)
        r = np.linspace(dr, rmax, self.end-self.start)

        for i, r_step in enumerate(r):
            integrand = 8 * math.pi * fq * np.sin(self.q * r_step)
            Gr[i] = np.trapz(integrand, self.s)

        return r, Gr

    def low_r_correction(self, Gr, nd, r, r_cut, scale_factor = 1):
        number_density_line = -4 * math.pi * nd * r * scale_factor
        Gr_low_r = np.where(r < r_cut, Gr, 0)
        Gr = np.where(r < r_cut, number_density_line, Gr)
        return Gr, Gr_low_r


    def cut_Gr_spherical(self, Gr, r_values, diameter):
        Gr = Gr * (1 - ((3 / 2) * (r_values / diameter)) + (0.5 * (r_values / diameter) ** 3)) * np.exp(-((r_values * 0.2 ** 2) / 2))
        return Gr

    def inverse_fourier_transform(self, Gr, r_values, fq_direct):
        # Initialize S(Q) to zeros
        fq_inverse = np.zeros_like(self.q)

        # Perform the inverse Fourier transform
        for i, dq in enumerate(self.q):
            integrand = Gr * np.sin(dq * r_values)
            fq_inverse[i] = np.trapz(integrand, r_values)

        if sum(fq_damp) != 0:
            fq_inverse = sum(fq_direct) / sum(fq_inverse) * fq_inverse
            return fq_inverse
        else:
            return fq_inverse

    def calculate_IQ(self, fq, damping):
        iq = fq * self.N * self.fq_sq
        iq = iq / (self.s*np.exp(-self.s2*damping))
        iq = iq + self.autofit
        return iq

    def plot_results(self, fq, Gr0, r, Gr1):
        f, ax = plt.subplots(1, 3, figsize=(14, 5))

        # Plotting I(Q) and Fit
        line1, = ax[0].plot(self.q, self.autofit)
        line2, = ax[0].plot(self.q, self.iq)
        ax[0].legend([line1, line2], ["Fit", "I(Q) $Fe_{3}O_{4}$"])
        #ax[0].text(5, 600000, 'N: ' f'{int(self.N)}')
        ax[0].set_xlabel("Q ($\AA^{-1}$)")
        ax[0].set_ylabel("Intensity")
        ax[0].title.set_text('Fitting I(Q)')

        # Plotting S(Q)
        line3, = ax[1].plot(self.q, fq)
        #line4, = ax[1].plot(self.q, self.phi_q_cut)
        ax[1].set_xlabel("Q ($\AA^{-1}$)")
        ax[1].set_ylabel("S(Q)")
        ax[1].legend([line3, None], ["$\phi(Q)$ damped",""])
        ax[1].title.set_text('Calculating $\phi(Q)$')

        # Plotting G(r)
        line5, = ax[2].plot(r, Gr0)
        line6, = ax[2].plot(r, Gr1)
        ax[2].set_xlabel("r ($\AA$)")
        ax[2].set_ylabel("G(r)")
        ax[2].set_xlim([0, 10])
        ax[2].title.set_text('Calculating G(r)')

        plt.savefig('I(Q)_S(Q)_G(r).jpeg', format='jpeg')

        f.tight_layout()
        plt.show()

    def save_to_csv(self, Gr, name, separator):
        if os.path.isfile(self.file_path):
    # Get the directory part of the file path
          directory = os.path.dirname(self.file_path)
          Gr = pd.DataFrame(np.transpose(np.array(Gr)))
          Gr.to_csv(f'{directory}/'f'{name}.csv', sep=separator, index=False, header=False)
          return None


In [ ]:
#@title 1.2. Main: Gerando G(r)
root_path = '/content/drive/MyDrive/CNPEM/Jacutingaita/Analises/PDF'  # Replace with your actual file path
file_path = '/Corning/ROI8/ROI8.txt'
lobato_path = '/Lobato_2014.txt'
start = 100
end = 1600
ds = 0.001147
nd = 0.05

Elements = {
    1: [14, 1],
    2: [8, 2],
    3: False,
    4: False,
    5: False
}

data_processor = DataProcessor(root_path + file_path, root_path + lobato_path, start, end, ds, Elements)
data_processor.calculate_Lobato_Factors()
data_processor.calculate_fq_gq()

_iq = data_processor.iq

data_processor.calculate_N_and_parameters()
_sq, _fq = data_processor.calculate_SQ_PhiQ(_iq , 0.3)
_r, _Gr = data_processor.calculate_Gr(_fq, rmax = 100, dr = 0.01)
_Gr, _ = data_processor.low_r_correction(_Gr, nd, _r, r_cut = 0.5)

df1 = pd.read_csv(root_path + '/Corning/Carbon_Grid/Carbon_Grid.csv')
_iq_OA = df1.values
_iq_OA = _iq_OA.reshape(_iq_OA.shape[0])
_iq_OA = _iq_OA[start:end]

data_processor.calculate_N_and_parameters()
_sq, _fq_OA_less = data_processor.calculate_SQ_PhiQ(_iq-_iq_OA, 0.3)
_r, _Gr_OA_less = data_processor.calculate_Gr(_fq_OA_less, rmax = 100, dr = 0.01)

"""
data_processor.calculate_N_and_parameters()
_sq, _fq_OA_less_beamless = data_processor.calculate_SQ_PhiQ(_iq -_iq_OA - _iq_beam, 0.3)
_r, _Gr_OA_less_beamless = data_processor.calculate_Gr(_fq_OA_less_beamless, rmax = 100, dr = 0.01)

_Gr, _ = data_processor.low_r_correction(_Gr, nd, _r, r_cut = 1.7)
_Gr_OA_less, _ = data_processor.low_r_correction(_Gr_OA_less, nd, _r, r_cut = 1.7)
_Gr_OA_less_beamless, _ = data_processor.low_r_correction(_Gr_OA_less_beamless, nd, _r, r_cut = 1.7)

path = '/content/drive/MyDrive/CNPEM/Jacutingaita/Analises/PDF/Fe3O4-Oleyl-OH/Fe3O4_Gr_calculated.dat'

_rbulk = pd.read_csv(path, sep =' ', dtype=float).values[:,0]
_Grbulk = pd.read_csv(path, sep =' ', dtype=float).values[:,1]
_Grnano = data_processor.cut_Gr_spherical(Grbulk, rbulk, 40)
"""
data_processor.plot_results(_fq, _Gr, _r, _Gr)


In [ ]:
#@title 1.3. Import de valores do eRDF Analyser para validação
df1 = pd.read_csv(file_path + '/Fe3O4-Oleyl-OH/ROI1/Data_r.csv', header = 0)
r1 = np.array(df1['r'])
Gr1 = np.array(df1['Gr'])

#df2 = pd.read_csv(file_path + '/Olavo/Data_q.csv', header = 0)
#q1 = np.array(df2['q'])

#phiqdamp1 = np.array(df2['phiq_damp'])
#Autofit1 = np.array(df2['Autofit'])
#gq1 = np.array(df2['gq'])
#fq_sq1 = np.array(df2['fq_sq'])
#print(q1[:-1]-q)


#df3 = pd.read_csv(file_path + 'Olavo/f1.csv', header = None)
#f1 = np.array(df3.values)
#f1 = f1.T
#f1 = f1.reshape(1843,)
#rint(Lobato_Factors[0]-f1[:-1])

#df4 = pd.read_csv(file_path + 'Olavo/q.csv', header = None)
#q1 = np.array(df4.values)

plt.plot(r, Gr)
plt.plot(r1, Gr1)
plt.xlim(0,20)
